# Kubernetes Workloads

In the world of Kubernetes, the term "workload" refers to the heart of your application - the components that you want to run, manage, and scale within the Kubernetes cluster. These workloads could be web servers, databases, microservices, or any other software that you need to deploy and maintain efficiently.

Workloads can be divided into pods and workload resources.

## Pods Overview

- A Kubernetes Pod is the smallest deployable unit, and it serves as the fundamental building block for your workloads. A Pod can contain one or more containers that run together as a single unit.

- It's essential to understand that Pods are designed to be ephemeral. If a Pod fails, Kubernetes can easily recreate it on another node in the cluster. This ephemeral nature ensures resilience and reliability in your applications.

- Kubernetes takes the responsibility of managing and rescheduling Pods in the case of node failures or Pod failures. Cluster administrators typically focus on monitoring and ensuring the overall health of the cluster.

## Workload Resources Overview

In Kubernetes, managing individual Pods manually can be inefficient and error-prone. This is where *Workload Resources* come into play. 

> Workload Resources are higher-level abstractions that simplify the deployment, scaling, and lifecycle management of Pods. They provide a declarative approach to managing your application, where you specify what you want, and Kubernetes takes care of the rest.

Here are some common Workload Resources:

- *Deployment*: Use Deployments for deploying stateless applications that need scaling and rolling updates. Deployments ensure that a specified number of replicas (Pods) are running at all times.

- *StatefulSet*: When you have stateful applications with unique identities (e.g., databases), StatefulSets are the go-to choice. They provide stable network identities and persistent storage.

- *DaemonSet*: For tasks that need to run on every node in the cluster (e.g., logging agents or monitoring tools), DaemonSets ensure that one Pod runs on each node.

- *Job and CronJob*: Jobs are used for running batch processes or one-off tasks, while CronJobs allow you to schedule recurring tasks in a cron-like fashion.

## Pods

A Pod in Kubernetes is a fundamental concept and serves as the smallest deployable unit. It's essentially a group of one or more containers that share a common context and a specification for running those containers. Think of Pods as similar to `docker-compose`, allowing you to logically group related containers together.

### Shared Context

Pods provide a shared context for containers, which includes:

- **Shared Storage**: Containers within a Pod can share the same storage volume, making it easy for them to exchange data

- **Shared Network Resources**: Containers in a Pod share the same network namespace, often having the same IP address and port space. This allows them to communicate with each other using `localhost`.

- **Linux Namespaces**: Pods use Linux namespaces to isolate containers within the same Pod, providing a level of process and network isolation.

Importantly, even within a Pod, individual applications can be further isolated as needed.

> A Pod is the minimal deployment unit in Kubernetes, which means containers cannot be deployed on their own. They always reside within a Pod.

### Lifecycle

Pods remain on the node where they are scheduled until they are terminated (according to the restart policy in case of failures) or deleted. 

> Unlike some other container orchestration platforms, Kubernetes Pods are never moved across nodes; instead, they are recreated if needed.

Key features include:

- Pods cannot initiate self-healing, meaning they don't attempt to restart themselves. The responsibility for restarting Pods lies with the appropriate Workload Resources or the cluster administrator.

- Kubernetes uses `kubelet` to automatically restart failed containers within Pods

- When a Pod is terminated, related resources like volumes are also deleted, unless specified otherwise

#### Pod Phases

A Pod can be in one of several phases:

- **Pending**: The Pod has been accepted by the Kubernetes cluster, but one or more containers may not have started yet, or the Pod is waiting for node scheduling.

- **Running**: At least one container within the Pod is running or being restarted

- **Succeeded**: All containers in the Pod have completed their tasks successfully, and no restart is required

- **Failed**: At least one container has failed and was terminated

- **Unknown**: The state of the Pod cannot be determined, typically due to communication errors with the Node

### Container states

Kubernetes also monitors the state of individual containers within each Pod.

Containers can be in one of three states:

- **Waiting**: This state indicates that the container is waiting for something to happen. It's often seen when a container is downloading its image or pulling secrets required for its operation. The reason for this state is provided for monitoring purposes.

- **Running**: This state means the container is actively executing its tasks and is in a healthy operational state

- **Terminated**: Containers can be in this state for various reasons. It may be due to a successful termination of the container's task, or it might indicate a failure. When a container terminates, Kubernetes provides the reason for termination and an exit code, which can be invaluable for monitoring and debugging.

### Single-Container Pods

Typically, Pods run with a single container, and a Pod can be seen as a wrapper for that container. Examples include:

- A FastAPI server receiving requests and saving data to a shared database
- A Docker container that receives image classification requests and forwards them to a classification service

### Multiple-Container Pods

Multi-container Pods are used in more complex scenarios where multiple tightly coupled containers work together as a cohesive unit. 

<p align=center><img src=images/pod.svg width=350></p>

This setup is ideal for scenarios like:

- Training multiple machine learning models, where containers perform tasks like data transformation, model training, model deployment, and serving
- Implementing a public data-serving container alongside an internal data-writing container with shared storage

> Multi-container Pods are scheduled on the same "logical host" due to their tight coupling.

To view Pods in your cluster, you can use the `kubectl get pod` command. By default, it shows Pods only in the default namespace. To see all Pods in the cluster, add the `-A` flag.

If you haven't done so already, you can create a local cluster with `minikube start`. This section will demonstrate how to deploy Pods in the next part.

In [1]:
!kubectl get pod -A

NAMESPACE              NAME                                         READY   STATUS      RESTARTS         AGE
default                hello-pod                                    1/1     Running     0                3h10m
default                ubuntu-pod                                   1/1     Running     0                3h2m
ingress-nginx          ingress-nginx-admission-create-dv2nn         0/1     Completed   0                21h
ingress-nginx          ingress-nginx-admission-patch-hxjtf          0/1     Completed   1                21h
ingress-nginx          ingress-nginx-controller-7799c6795f-b7zkl    1/1     Running     1 (4h1m ago)     21h
kube-system            coredns-5d78c9869d-4nqdp                     1/1     Running     7 (4h1m ago)     2d2h
kube-system            etcd-minikube                                1/1     Running     7 (4h1m ago)     2d2h
kube-system            kube-apiserver-minikube                      1/1     Running     7 (4h1m ago)     2d2h
kube-system  

We can see we already have some Pods. This is because `minikube` deploys some Pods by default. In the next section, we will look at how to deploy Pods.

### Defining Pods

In Kubernetes, you can define Pods imperatively (step by step) or declaratively using `.yaml` files. However, specifying bare Pods directly is generally discouraged because it leaves them without the necessary mechanisms for self-recovery and management. Instead, it's recommended to use higher-level abstractions like Deployments or ReplicaSets to manage Pods and ensure their availability.

While it's possible to specify a bare Pod directly, it's important to understand that doing so may not be suitable for most production scenarios. Below is an example of specifying a bare Pod for reference:

``` yaml
apiVersion: v1
kind: Pod
metadata:
  name: pod1
  labels:
    tier: frontend
spec:
  containers:
  - name: hello1
    image: gcr.io/google-samples/hello-app:2.0
```

In this example, we've defined a Pod named `pod1` with a single container called `hello1` using a specific Docker image. However, without higher-level controllers like Deployments or ReplicaSets, this Pod won't have the ability to recover from failures or scale automatically.

In practice, for robust and scalable application deployments, it's recommended to use the appropriate Workload Resources, which we'll explore in the next section, to manage Pods effectively.

Now it's time to try running some `kubectl` commands yourself. Follow the instructions to run some important Pods commands:

In [2]:
# Observe the pods in the default namespace
!kubectl ###Your command here

NAME         READY   STATUS    RESTARTS   AGE
hello-pod    1/1     Running   0          3h13m
ubuntu-pod   1/1     Running   0          3h6m


In [4]:
# Spin up the pod corresponding to the single-pod configuration above
!kubectl ##Your command here

pod/pod1 created


In [5]:
# Observe the pods in the default namespace again
!kubectl ##Your command here

NAME         READY   STATUS    RESTARTS   AGE
hello-pod    1/1     Running   0          3h14m
pod1         1/1     Running   0          17s
ubuntu-pod   1/1     Running   0          3h6m


In [6]:
# Delete the pod created above
!kubectl ##Your command here

pod "pod1" deleted


In [7]:
# observe the resources in the default namespace again 
!kubectl ##Your command here

NAME         READY   STATUS    RESTARTS   AGE
hello-pod    1/1     Running   0          3h15m
ubuntu-pod   1/1     Running   0          3h7m


Unlike the case in the last notebook, the pod disappears here. This is because, contrary to the case in the last notebook, no instructions were provided here to keep the pod alive. The pod is 'bare', without any resource to keep it running after it fails. 

Keeping it 'alive' is one of the desired states that can be specified in the config file, and it can be achieved with Deployment or Replica Set. We will look at more options in the next section.

## Workload Resources

Before diving into specific Workload Resources, let's cover some important concepts that apply to all of them:

- Each Workload Resource uses the `.spec.template` field to specify how to create a Pod

- The template for a Workload Resource is essentially the same as a Pod configuration, except for the `kind` and `apiVersion` fields

- Workload Resources also have a `.spec.selector` field, which specifies which Pods are managed by the Workload Resource

- The `.spec.selector` can employ matches on defined labels, allowing the Workload Resource to manage Pods even if they are defined in a separate configuration file

Workload Resources can be implemented using different controllers, including **ReplicaSets**, **Deployment**, **DaemonSet**, and **Jobs**. In a future lesson, we will explore another type of Workload Resource called *StatefulSets*.

### ReplicaSet

>  ReplicaSet is a controller that ensures a stable set of replicated Pods running at any given time. It maintains a specified number of replicas of Pods.

#### Acquiring Pods

ReplicaSet acquires Pods using the `metadata.ownerReferences` field and matching the .`spec.selector`. Here's how it works:

- Each Pod has an `metadata.ownerReferences` field that is automatically added by Kubernetes. This field specifies who manages the Pod, which can be another controller.

- If a Pod has no 'owner' (e.g., it's a bare Pod) or its owner is not another controller, and the `.spec.selector` fields match, then the Pod is acquired by the ReplicaSet.

> This acquisition process is similar for other Workload Resources.

#### Using ReplicaSets

In general, it's recommended to use higher-level controllers like Deployments for managing Pods and ReplicaSets. Deployments manage ReplicaSets and provide declarative updates to Pods.

ReplicaSets may be employed when:

- Custom update orchestration is required
- The configuration file will not be updated

However, it's worth noting that Deployments offer a higher level of abstraction and ease of use.

For more detailed information on ReplicaSets, check [here](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/).

### Deployment

> Deployment is a controller that provides declarative updates for Pods and ReplicaSets.

Consider the example config below, and attempt to decipher the meaning of each field:

``` yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80
```

In this example, we instruct the resource to ensure that at least three Pods are running at all times. These Pods are identified using the `selector.matchLabels`, which searches the template for the label with the key `app` and value `nginx`.

#### Hands-On

1. Create a `.yaml` file with the provided configuration
2. Use the appropriate `kubectl` command to apply the `.yaml` file
3. Observe the number of Pods created
4. Delete one of the Pods
5. Observe the number of Pods again

In [1]:
# run the .yaml file
!kubectl 

deployment.apps/nginx-deployment created


In [2]:
# Observe how many pods you have
!kubectl

NAME                                READY   STATUS              RESTARTS   AGE
nginx-deployment-66b6c48dd5-7qpvr   0/1     ContainerCreating   0          17s
nginx-deployment-66b6c48dd5-gc7z7   0/1     ContainerCreating   0          17s
nginx-deployment-66b6c48dd5-vzsx2   0/1     ContainerCreating   0          17s


In [1]:
# Delete one of the pods
!kubectl delete pod nginx-deployment-66b6c48dd5-vzsx2

pod "nginx-deployment-66b6c48dd5-vzsx2" deleted


In [2]:
# Observe how many pods you have again
!kubectl get pod

NAME                                READY   STATUS    RESTARTS        AGE
nginx-deployment-66b6c48dd5-7qpvr   1/1     Running   1 (8m15s ago)   8h
nginx-deployment-66b6c48dd5-fsck5   1/1     Running   0               40s
nginx-deployment-66b6c48dd5-gc7z7   1/1     Running   1 (8m15s ago)   8h


After deleting a Pod, you should see that a new Pod has been created by the Deployment to maintain the desired number of replicas.

### DaemonSet

> DaemonSet is a controller that ensures a Pod is deployed on all Nodes in the cluster. Each Node gets one instance of the Pod.

In the examples thus far, we had one pod per node; therefore, if a node is removed, the number of pods will decrease.

#### Node Selection in DaemonSets

One distinctive feature of DaemonSets is that they ensure a single pod runs on each node in the cluster. The `replicas` field, commonly used in Deployments, is not present in DaemonSet configurations because there is always one pod per node.

Generally, Kubernetes automatically handles the assignment of pods to nodes. However, there are cases where you might want to influence this process:

- Ensuring that a pod runs on a node with specific hardware characteristics (e.g., a node with SSD storage)
- Co-locating pods from different services on the same node if they frequently communicate

Kubernetes provides a mechanism for node selection through the use of labels and node selectors. Nodes can be labeled with various attributes, and then, in the DaemonSet configuration, you can specify `nodeSelector` rules to determine which nodes receive the DaemonSet pods.

Here's an example DaemonSet configuration `YAML`:

``` yaml 
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: fluentd-elasticsearch
spec:
  selector:
    matchLabels:
      name: fluentd-elasticsearch
  template:
    metadata:
      labels:
        name: fluentd-elasticsearch
    spec:
      containers:
      - name: fluentd-elasticsearch
        image: quay.io/fluentd_elasticsearch/fluentd:v2.5.2
        resources:
          limits:
            memory: 200Mi
          requests:
            cpu: 100m
            memory: 200Mi
      terminationGracePeriodSeconds: 30
```

In this example, a DaemonSet named `fluentd-elasticsearch` ensures that a pod running the specified container image (`quay.io/fluentd_elasticsearch/fluentd:v2.5.2`) is deployed to every node in the cluster.

Before proceeding, let's briefly go over memory resource management in Kubernetes. In the provided DaemonSet configuration, you'll notice a new field within the `template.spec.container` section called `resources`. This field allows you to set resource limits and requests for the container.

- `limits`: These values represent the maximum resource capacities that are allotted to a pod. If a process within the pod consumes more than the specified limit (e.g., exceeding 200MB of RAM), Kubernetes will take action, potentially restarting the pod to enforce resource limits.

- `requests`: These values represent the minimum resource capacities that are guaranteed to be available to the pod. In the example configuration, the pod is guaranteed a minimum of 200MB of RAM and 100 milicores (1/1000th of a core). Kubernetes uses these requests for scheduling and placement decisions.

For more detailed information on container resource management, you can refer to the [official Kubernetes documentation on managing container resources](https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/).

### Jobs

Jobs create one or more Pods and repeatedly attempt to execute them until a specified number successfully terminates. Jobs are useful when you need to ensure the successful execution of a task or when you want to run the same task in parallel multiple times.

Here's an example of a Job workload that calculates the value of π:

``` yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: pi
spec:
  template:
    spec:
      containers:
      - name: pi
        image: perl
        command: ["perl",  "-Mbignum=bpi", "-wle", "print bpi(2000)"]
      restartPolicy: Never
  backoffLimit: 4
```

#### Specification

For Jobs, the standard fields are necessary, in addition to the following fields:

- `.spec.restartPolicy`: Can be either `Never` or `OnFailure` (default). It specifies what should happen when a Pod completes its task.

- .`spec.completions`: The number of Pods that should successfully complete before the Job is considered successful

- `.spec.parallelism`: The number of Pods to run simultaneously. This allows you to control parallelism in different ways.

Using the combination of `.spec.completions` and `.spec.parallelism`, you can construct different levels of parallelism:

- **Non-parallel**: If you specify `.spec.completions=1`, only one Job will be created at a time, and a new one will only start after the previous one fails

- **Parallel with a fixed completion count**: By setting `.spec.completions=N`, you can run at most N parallel Jobs at any given time. If a Pod or Job fails, the controller will reschedule it to maintain the desired count.

- **Parallel with a work queue**: This configuration is achieved by specifying `.spec.completions=1` and `.spec.parallelism=N`. In this mode, N Pods will run concurrently, but the execution of additional Pods continues until all tasks are completed. This approach is useful when you want Pods to communicate directly with each other for improved efficiency.

> It's important to note that the default mode is non-parallel, and the default values for `.spec.completions` and `.spec.parallelism `are both set to 1.

Additionally, you can specify the [completion mode](https://kubernetes.io/docs/concepts/workloads/controllers/job/#completion-mode), which allows you to modify the behaviour of the Pods upon termination.

Here are some key points to keep in mind:

- Depending on the settings, if `.spec.completions=N` is achieved, the Job is considered successful

- Until that point, Kubernetes will attempt to recreate Pods associated with the Job N times, as specified by `.spec.completions`

- Exponential back-off delay is applied in case of failures, starting with a retry after 10 seconds, then 20 seconds, and doubling each time, capped at a maximum backoff period of 6 minutes.

- `.spec.activeDeadlineSeconds`: This field defines the maximum duration (in seconds) that the entire Job is allowed to run. Once this duration is reached, all Pods associated with the Job are terminated, regardless of their completion status.
    
#### Cleaning Up

By default, completed Jobs are not automatically removed from the cluster. This is because you may need to check logs or the status of completed Jobs.

However, to manage resources effectively, you can use TTL (time to live) to specify when the Job and its associated Pods should be removed from the cluster. TTL can be set using the `.spec.ttlSecondsAfterFinished` field.

As mentioned, Jobs will terminate when they successfully execute a number of Pods. To repeat that operation, you would need to `apply` a Kubernetes object to re-run the job. Fortunately, there is an easier approach to generate Jobs periodically with the desired frequency, namely *Cronjobs*.

### CronJobs

CronJobs create Jobs on a recurring schedule. You can specify the schedule in the `spec.schedule` field, similar to a cron expression.

``` yaml
apiVersion: batch/v1
kind: CronJob
metadata:
  name: hello
spec:
  schedule: "*/1 * * * *"
  jobTemplate:
    spec:
      template:
        spec:
          containers:
          - name: hello
            image: busybox
            imagePullPolicy: IfNotPresent
            command:
            - /bin/sh
            - -c
            - date; echo Hello from the Kubernetes cluster
          restartPolicy: OnFailure
```

This CronJob is scheduled to run according to the specified cron schedule, `*/1 * * * *`.

This schedule means that the CronJob runs every minute, as indicated by the `*/1 `in the first field, which represents minutes. The `*` in the other fields means "every" for hours, days of the month, months, and days of the week, respectively.

> A CronJob specifies its schedule using the cron syntax, which consists of five fields: minute, hour, day of the month, month, and day of the week. You can use wildcards (`*`) and ranges to define flexible schedules. For example, a CronJob that runs every day at midnight would have the schedule `0 0 * * *`.

## Clean Up

Before wrapping up this lesson, let's make sure we clean up all the resources we have been provisioning this lesson. Run the following commands to make sure everything we provision is deleted (you should delete the two  `YAML`s you have created, one for the Pod and one for the `nginx` Deployment):

In [ ]:
!kubectl delete -f <your-yaml-file>

## Key Takeaways

- Kubernetes workloads refer to applications and services running on a Kubernetes cluster. They can be categorized into two main components: Pods and Workload Resources.
- Pods are the smallest deployable units in Kubernetes. They group one or more containers, sharing a common context.
- Workload Resources are higher-level abstractions for managing Pods' lifecycles. They include ReplicaSets, Deployments, DaemonSets, Jobs, and CronJobs. Workload Resources simplify the management of Pods, enabling declarative updates and scaling.
- ReplicaSets maintain a specified number of replicated Pods. They create and delete Pods based on the desired replica count.
- Deployments provide declarative updates for Pods and ReplicaSets. They enable rolling updates and rollbacks, ensuring application availability during changes.
- DaemonSets ensure that a Pod runs on every node in the cluster. They are suitable for monitoring or managing services on each node.
- Jobs create one or more Pods to complete a task. They can run in parallel, and their completion behavior can be customized.
- CronJobs automate the creation of Jobs on a schedule. They allow recurring tasks to be executed at specified intervals.